In [1]:
#Befehle, die verschiedene Funktionen von Spacy laden. Basis für die Programmierung.
import spacy
from collections import Counter
from spacy import displacy
import pandas as pd
from spacytextblob.spacytextblob import SpacyTextBlob
from pathlib import Path
import os

In [2]:
#Englisches Sprachpaket small und Spacytextblobpipe auf nlp laden
nlp = spacy.load ('en_core_web_sm')
nlp.add_pipe('spacytextblob')
pathlist = Path("Songs").glob('**/*.txt')
count = 0

In [3]:
# Funktion, die den Text normalisiert: Entfernt alle Satz- und Lehrzeichen, wandelt konjugierte Verben in Stammform um.
# Gibt am Ende den normalisierten Text zurück. Funktion kann auf jedem beliebigen Text aufgerufen werden

def normalize_ws (text):
    norm_text = []
    for token in text:
            if not token.is_punct and not token.is_space:
                    norm_text.append(token.lemma_.lower())
    return ' '.join(norm_text)

In [4]:
for path in pathlist:
    #Textdatei wird eingelesen, Zeilenumbrüche werden entfernt
    text = open(path).read()
    text = text.replace("\n", " ").strip()
    
    #Text wird in tokenisierten NLP Text umgewandelt
    doc = nlp(text)
    
    #Path wird in String umgewandelt um den Dateinamen zuordnen zu können -funktioniert nicht
    f_name, f_ext = os.path.splitext(path)
    song_name = []
    song_name.append(f_name)
    
    # Das "DOC" wird mit dem Funktionsaufruf "normalize_ws" normalisiert und abgespeichert
    normalized_ws_text = normalize_ws(doc)
    # Das normalisierte DOC muss wieder in einen NLP Text umgewandelt werden
    normalized_ws_doc = nlp(normalized_ws_text)
    # Alle Nouns werden aus dem normalisierten Text herausgefiltert:
    nouns_ws = [ token.text for token in normalized_ws_doc if token.pos_ == 'NOUN']
    # Alle Verben werden aus dem normalisierten Text herausgefiltert:
    verbs_ws = [ token.text for token in normalized_ws_doc if token.pos_ == 'VERB']

    #Anzahl der Token umgewandelt in String und dann in Token
    number = len(nlp(normalized_ws_doc)) 
    number = str(number)
    number = nlp(number)
    
    # Berechnung der 10 am häufigsten vorkommenden Nouns
    ten_nouns = Counter(nouns_ws).most_common(10)
    # Berechnung der 10 am häufigsten vorkommenden Verben
    ten_verbs = Counter(verbs_ws).most_common(10)
    #Arrays werden initialisiert
    song_sent_score = []
    song_sent_label = []
    song_sent_subjectivity = []
    total_pos = []
    total_neg = []
    positive_words = []
    negative_words = []

    #Sentiment und Subjectivity wird dem Lied zugeordnet
    sentiment = doc._.blob.polarity
    sentiment = round(sentiment,2)
    subjectivity = doc._.blob.subjectivity
    subjectivity = round(subjectivity,2)

    #Sentimentscore wird verwendet um das Lied als positives oder negatives einzustufen
    if sentiment > 0:
          sent_label = "Positiv"
    else:
      sent_label = "Negativ"

    #Resultate werden den entsprechenden Kategorien zugeordnet
    song_sent_label.append(sent_label)
    song_sent_score.append(sentiment)
    song_sent_subjectivity.append(subjectivity)

    #Einzelne Tupel (Token + Sentiment) werden als positives oder negatives Wort geteilt
    for x in doc._.blob.sentiment_assessments.assessments:
      if x[1] > 0:
        positive_words.append(x[0][0])
      elif x[1] < 0:
        negative_words.append(x[0][0])
      else:
        pass

    #Resultate werden den entsprechenden Kategorien zugeordnet
    total_pos.append(', '.join(set(positive_words)))
    total_neg.append(', '.join(set(negative_words)))
    
    #Fehlende Arrays werden erstellt
    total_nouns = []
    total_verbs = []
    total_amount = []

    #Listen werden mit Panda als Basisdatei eingelesen
    liste = pd.read_csv("liste.csv")
    liste1 = pd.read_csv("liste1.csv")
    liste2 = pd.read_csv("liste2.csv")


    #Liste wird mit Daten gefüllt
    liste1 ["Häufigste Nouns"]= ten_nouns
    liste2 ["Häufigste Verben"]= ten_verbs


    #Liste 1 wird mit Daten gefüllt
    liste["Dateiname"]= song_name
    liste["Anzahl aller Token"] = number
    liste["Sentiment Score"] = song_sent_score
    liste["Sentiment Label"] = song_sent_label
    liste["Sentiment Subjectivity"] = song_sent_subjectivity
    liste["Positive Wörter"] = total_pos
    liste["Negative Wörter"] = total_neg
    
    strcount = str(count)

    #Listen werden als CSV gespeichert
    liste.to_csv("Sentiment\sentiment"+strcount+".csv")
    liste1.to_csv("Nouns\liste1"+strcount+".csv")
    liste2.to_csv("Verben\liste2"+strcount+".csv")
    
    count = count+1

In [5]:
# Ausgabe zur Kontrolle
#print(nouns_ws)
#print(verbs_ws)
#print(words_ws)
#print (doc)
#print (normalized_ws_doc)
#print(ten_nouns)
#print(ten_verbs)
#print(ten_words)